In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import importlib

# List of required packages
required_packages = [
    "torch",
    "transformers",
    "pandas",
    "numpy",
    "immutabledict", 
    "sentencepiece",
]

# Check if each package is installed
for package in required_packages:
    try:
        module = importlib.import_module(package)
        version = getattr(module, "__version__", "Version not found")
        print(f"{package} is installed. Version: {version}")
    except ImportError:
        print(f"{package} is NOT installed. Please install it using '!pip install {package}'")


torch is installed. Version: 2.4.0
transformers is installed. Version: 4.45.1
pandas is installed. Version: 2.2.3
numpy is installed. Version: 1.26.4
immutabledict is installed. Version: 4.2.0
sentencepiece is installed. Version: 0.2.0


In [3]:
import torch

# Check available GPUs
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs for inference.")
else:
    print("Only one GPU is available. Using single GPU.")


Using 2 GPUs for inference.


## Load Model  

In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Define the model directory
weights_dir = '/kaggle/input/qwen2.5/transformers/3b-instruct/1'

# Load the model and tokenizer from the specified directory
tokenizer = AutoTokenizer.from_pretrained(weights_dir)
model = AutoModelForCausalLM.from_pretrained(weights_dir,ignore_mismatched_sizes=True,device_map="auto")


prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
